In [ ]:
import pandas as pd  
import numpy as np  
import os  
import pyodbc  
import pandasql as ps  
from datetime import datetime
from openpyxl import load_workbook

# Paths to all cleaned stock files  
file_paths = {  
    'CJ_Stock': r'D:\Data for Stock Report\cleaned_CJ_Stock_Report.xlsx',  
    'Daily_SO': r'D:\Data for Stock Report\appended_cleaned_SellOut.xlsx',
    'Access_PO': r'D:\Data for Stock Report\cleaned_PO_pending_other.xlsx',  
    'Daily_Stock_DC': r'D:\Data for Stock Report\cleaned_DC_daily_stock.xlsx',
    'Master_Owner&LT': r'C:\Users\Thanawit C\OneDrive - Sahamit Product Co.,Ltd\Data for Stock Report\COPY_MasterLeadTime.xlsx'  
}  

# Load Excel files into DataFrames
def load_data(file_paths):
    dataframes = {}
    for name, path in file_paths.items():
        try:
            if name == "Master_Owner&LT":
                # Load the specified sheet
                owner_scm_df = pd.read_excel(path, sheet_name='All_Product', header=1) 
                
                # Select specific columns
                owner_scm_df_selected = owner_scm_df[['SHM_Item', 'CJ_Item', 'OwnerSCM', 'Base Lead Time (Days)']]
                owner_scm_df_selected.rename(columns={'Base Lead Time (Days)': 'LeadTime'}, inplace=True)

                # Cast CJ_Item to str
                owner_scm_df_selected['CJ_Item'] = owner_scm_df_selected['CJ_Item'].astype(str)
                # Clean CJ_Item column
                owner_scm_df_selected['CJ_Item'] = owner_scm_df_selected['CJ_Item'].fillna('')
                owner_scm_df_selected['CJ_Item'] = owner_scm_df_selected['CJ_Item'].apply(lambda x: x.split('.')[0] if '.0' in str(x) else x)

                # Store the processed DataFrame
                dataframes[name] = owner_scm_df_selected
            else:
                # Load other sheets  
                sheet_name = {
                    'CJ_Stock': 'CJ Stock',
                    'Daily_SO': 'Pivot SO',
                    'Access_PO': 'Pivot All PO pending',  
                    'Daily_Stock_DC': 'Pivot_DC_stock'
                }.get(name)  

                dataframes[name] = pd.read_excel(path, sheet_name=sheet_name, parse_dates=False)
        except Exception as e:  
            print(f"Error loading {name}: {e}")
    return dataframes

# Connect to Access database and fetch product details  
def load_access_data():  
    access_db_path = r'D:\DataBase Access\SHM_TMS_001_Master_Copy.accdb'  
    conn_str = (  
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'  
        f'DBQ={access_db_path};'  
    )  
    try:  
        conn = pyodbc.connect(conn_str)  
        print("Connection successful")  

        query = """  
        SELECT t1.CJ_Item,
               t1.SHM_Item,  
               t1.CJ_Description,  
               t1.Devision,  
               t1.[Group],  
               t1.PC_Cartons,  
               t1.CJ_Status,  
               t1.Cat,  
               t1.Sub_Cat,  
               t1.Brand,
               First_SO_Date,
               t2.Supplier_Code,
               t2.[Supplier Name]
          FROM tbl_master_003_product_list AS t1
          LEFT JOIN qry_Product_list AS t2
          ON t1.SHM_Item = t2.SHM_Item
        """

        query2 = """  
        SELECT 
            t2.Item,
            MIN(t2.Unit) AS Unit
        FROM (
            SELECT Item, Unit, [PO Date] AS po_date
            FROM tbl_AllPO_Details
            ) AS t2
        INNER JOIN (
            SELECT Item, MAX([PO Date]) AS max_date
            FROM tbl_AllPO_Details
            Group by Item
            ) AS t1
        ON t1.Item = t2.item AND t2.po_date = t1.max_date
        GROUP BY t2.Item
        """

        # Execute the queries
        access_df = pd.read_sql(query, conn)
        query2_df = pd.read_sql(query2, conn)

        # merge for the unit of purchase product info
        access_df2 = pd.merge(access_df, query2_df, left_on='SHM_Item', right_on='Item', how='left')

        conn.close()  
        print("Connection to Access database is closed.")  
        return access_df2
    
    except Exception as e:  
        print(f"Connection failed: {e}")  
        return pd.DataFrame(), pd.DataFrame()


# Convert 'CJ_Item' to string format in all DataFrames  
def convert_cj_item_to_string(dataframes, access_df2):
    def process_column(df, column):
        if column in df.columns:
            df[column] = df[column].astype(str).str.split('.').str[0]

    for df in dataframes.values():
        process_column(df, 'CJ_Item')
        process_column(df, 'SHM_Item')

    # Cast CJ_Item to str (when the access_df2 is a dataframe not dictionary)
    process_column(access_df2, 'CJ_Item')
    process_column(access_df2, 'SHM_Item') 

    return access_df2

# Merge all DataFrames  
def merge_dataframes(dfs, access_df):
    merged_df = access_df.merge(dfs['CJ_Stock'], on='CJ_Item', how='left')

    merged_df = merged_df.merge(dfs['Daily_SO'], on='CJ_Item', how='left')  

    merged_df = merged_df.merge(dfs['Access_PO'], on='SHM_Item', how='outer', suffixes =('', '_from-AccessPO'))

    merged_df = merged_df.merge(dfs['Daily_Stock_DC'], on='CJ_Item', how='left', suffixes=('', '_from-DailyDC'))  

    merged_df = merged_df.rename(columns={  
        'Division': 'Division_CJ_stock',  
        'Devision': 'Division_SHM'  
    })  

    # Create a new column for NPD Status by First_SO_Date, current logic = First SO date + 15 days
    today = pd.to_datetime(datetime.now().date())  
    merged_df['days_from_first_ATP'] = (today - pd.to_datetime(merged_df['First_SO_Date'])).dt.days  
    merged_df['NPD_Status'] = np.where(merged_df['days_from_first_ATP'] <= 15, 'NPD', '-')

    # Fill in missing values for descriptive columns  
    merged_df['Name'] = merged_df['Name'].fillna(merged_df['CJ_Description'])  
    merged_df['Category'] = merged_df['Category'].fillna(merged_df['Cat'])  
    merged_df['Subcate'] = merged_df['Subcate'].fillna(merged_df['Sub_Cat'])  
    
    return merged_df


# Fill NaN values in numeric columns with 0
def fill_na_with_zero(df):
    df[df.select_dtypes(include=[np.number]).columns] = df.select_dtypes(include=[np.number]).fillna(0)
    return df


# Create new column to sum ALL PO Pending
def calculate_totals(merged_df):
    dc_columns = [1, 2, 4]
    for dc in dc_columns:
        merged_df[f'Total-PO_qty_to_DC{dc}'] = (
            merged_df[f'PO_Qty_to_DC{dc}'])

        # Calculate %Ratio with error handling
        merged_df[f'%Ratio_AvgSalesQty90D_DC{dc}'] = (
            merged_df[f'DC{dc}_AvgSaleQty90D'] / merged_df['Total_AvgSaleQty90D'].replace(0, 1)
        ).replace([np.inf, -np.inf], 0)

    # Calculate Total Remain Stock
    merged_df['Remain_StockQty_AllDC'] = merged_df['DC1_Remain_StockQty'] + merged_df['DC2_Remain_StockQty'] + merged_df['DC4_Remain_StockQty']
    merged_df['Remain_StockValue_AllDC'] = merged_df['DC1_Remain_StockValue'] + merged_df['DC2_Remain_StockValue'] + merged_df['DC4_Remain_StockValue']

    # Calculate SO Qty
    for dc in dc_columns:
        merged_df[f'DC{dc}_SO_Last30D'] = round(merged_df['SO_Qty_last30D'] * merged_df[f'%Ratio_AvgSalesQty90D_DC{dc}'])
        merged_df[f'DC{dc}_SO_Last7D'] = round(merged_df['SO_Qty_last7D'] * merged_df[f'%Ratio_AvgSalesQty90D_DC{dc}'])

        merged_df[f'DC{dc}_AvgSaleQty30D'] = merged_df[f'DC{dc}_SO_Last30D'] / 30
        merged_df[f'DC{dc}_AvgSaleQty7D'] = merged_df[f'DC{dc}_SO_Last7D'] / 7

    merged_df['Total_AvgSaleQty30D'] = merged_df[[f'DC{dc}_AvgSaleQty30D' for dc in dc_columns]].sum(axis=1)
    merged_df['Total_AvgSaleQty7D'] = merged_df[[f'DC{dc}_AvgSaleQty7D' for dc in dc_columns]].sum(axis=1)
    return merged_df


# Simplify DOH calculation
def calculate_DOH(stock_qty, avg_qty):
    return np.where(
        (stock_qty != 0) & (avg_qty == 0), 365,
        np.where((stock_qty == 0) & (avg_qty == 0), 0, stock_qty / avg_qty)
    )

# Simplified DOH calculations for various stock locations
def apply_doh_calculations(merged_df):
    dc_list = ['DC1', 'DC2', 'DC4']
    max_doh_value = 1825
    current_date = pd.to_datetime(datetime.now().date())

    # Calculate DC DOH and DOH after PO for each DC
    for dc in dc_list:
        merged_df[f'Current_{dc}_DOH'] = calculate_DOH(
            merged_df[f'{dc}_Remain_StockQty'],
            merged_df[f'{dc}_AvgSaleQty90D']
        )
        merged_df[f'{dc}_DOH(Stock+PO)'] = calculate_DOH(
            merged_df[f'{dc}_Remain_StockQty'] + merged_df[f'Total-PO_qty_to_{dc}'],
            merged_df[f'{dc}_AvgSaleQty90D']
        )

    # Special columns for all DCs
    merged_df['Current_DOH_All_DC'] = calculate_DOH(
        merged_df['Remain_StockQty_AllDC'],
        merged_df['Total_AvgSaleQty90D']
    )
    merged_df['Total-PO_qty_to_DC'] = merged_df[[f'Total-PO_qty_to_{dc}' for dc in dc_list]].sum(axis=1)

    merged_df['Current_DOH(Stock+PO)_All_DC'] = calculate_DOH(
        merged_df['Remain_StockQty_AllDC'] + merged_df['Total-PO_qty_to_DC'],
        merged_df['Total_AvgSaleQty90D']
    )

    # Ensure all delivery date columns are datetime
    dc_date_columns = {
        'DC1': ['Min_del_date_to_DC1'],
        'DC2': ['Min_del_date_to_DC2'],
        'DC4': ['Min_del_date_to_DC4']
    }

    for dc, cols in dc_date_columns.items():
        for col in cols:
            data_cols = merged_df[col].replace(['', '0', 0, 'null', 'None', '#N/A'], pd.NaT)
            data_cols = pd.to_datetime(data_cols, errors='coerce')
            data_cols = data_cols.where(~(data_cols.dt.date == pd.Timestamp('1970-01-01').date()), pd.NaT)

            merged_df[col] = data_cols       

        # Calculate Min deld ate only if at least one column has a non-null value
        merged_df[f'Min_delivery_date_to_{dc}'] = merged_df[cols].min(axis=1)

    merged_df['Min_delivery_date_to_DC'] = merged_df[[f'Min_delivery_date_to_{dc}' for dc in dc_list]].min(axis=1)

    # Cap DOH values
    doh_columns = [
        'Current_DOH_All_DC', 'Current_DOH(Stock+PO)_All_DC'
    ] + [f'Current_{dc}_DOH' for dc in dc_list] + [f'{dc}_DOH(Stock+PO)' for dc in dc_list]

    for col in doh_columns:
        merged_df[col] = np.where(merged_df[col] > max_doh_value, np.inf, merged_df[col])

    # Initialize cover date columns
    cover_date_cols = [
        'Total_Store_cover_to_date', 'Stock_All_DC_Cover_to_date', 'Stock+PO_All_DC_Cover_to_date'
    ] + [f'{prefix}_{dc}_cover_to_date' for dc in dc_list for prefix in ['Store', 'Stock', 'Stock+PO']]

    for col in cover_date_cols:
        merged_df[col] = pd.NaT

    return merged_df, current_date, max_doh_value


# Cover date calculation function
def apply_cover_date_calculations(merged_df, current_date, max_doh_value):
    cover_date_map = {
        'Total_Store_cover_to_date': 'Total_DOHStore',
        'Stock_All_DC_Cover_to_date': 'Current_DOH_All_DC',
        'Stock+PO_All_DC_Cover_to_date': 'Current_DOH(Stock+PO)_All_DC',
        'Store_DC1_cover_to_date': 'DC1_DOHStore',
        'Stock_DC1_cover_to_date': 'Current_DC1_DOH',
        'Stock+PO_DC1_cover_to_date': 'DC1_DOH(Stock+PO)',
        'Store_DC2_cover_to_date': 'DC2_DOHStore',
        'Stock_DC2_cover_to_date': 'Current_DC2_DOH',
        'Stock+PO_DC2_cover_to_date': 'DC2_DOH(Stock+PO)',
        'Store_DC4_cover_to_date': 'DC4_DOHStore',
        'Stock_DC4_cover_to_date': 'Current_DC4_DOH',
        'Stock+PO_DC4_cover_to_date': 'DC4_DOH(Stock+PO)'
    }

    def valid_doh_to_date(doh_value):
        return pd.notnull(doh_value) and doh_value > 0 and not np.isinf(doh_value) and doh_value <= max_doh_value

    for index, row in merged_df.iterrows():
        for target_col, source_col in cover_date_map.items():
            doh_value = row[source_col]
            if valid_doh_to_date(doh_value):
                merged_df.at[index, target_col] = current_date + pd.to_timedelta(doh_value, unit='d')

    return merged_df


# Main function
def main():
    dataframes = load_data(file_paths)
    access_df2 = load_access_data()
    access_df2 = convert_cj_item_to_string(dataframes, access_df2)
    
    merged_df = merge_dataframes(dataframes, access_df2)

    # Merge with master Owner and lead time to get owner name and lead time
    merged_df = merged_df.merge(dataframes['Master_Owner&LT'],on=['SHM_Item'],how='left', suffixes = ('', '_from-LT'))
    merged_df = merged_df.fillna({'OwnerSCM': 'No data', 'LeadTime': 'No data'})

    merged_df = fill_na_with_zero(merged_df)
    merged_df = calculate_totals(merged_df)
    
    merged_df, current_date, max_doh_value = apply_doh_calculations(merged_df)
    merged_df = apply_cover_date_calculations(merged_df, current_date, max_doh_value)

   # Execute SQL query
    query = """
    SELECT Division_SHM,
        OwnerSCM,
        [Supplier Name],
        SHM_Item,
        CJ_Item,
        Name,
        Category,
        Brand,
        LeadTime,
        Status,
        [Group],
        Unit,
        PC_Cartons,
        First_SO_Date,
        NPD_Status,
        Total_ScmAssort,
        Total_OOSAssort,
        Total_CountOKROOS,
        Total_PercOOS,
        Total_StoreStockQty,
        Total_DOHStore,
        Total_Store_cover_to_date,
        Total_AvgSaleQty90D,
        Total_AvgSaleQty30D,
        Total_AvgSaleQty7D,
        SO_Qty_last7D,
        Remain_StockQty_AllDC,
        Current_DOH_All_DC,
        Stock_All_DC_Cover_to_date,
        [Total-PO_qty_to_DC],
        Min_delivery_date_to_DC,
        [Current_DOH(Stock+PO)_All_DC],
        [Stock+PO_All_DC_Cover_to_date],
        DC1_ScmAssort,
        DC1_OOSAssort,
        DC1_CountOKROOS,
        DC1_PercOOS,
        DC1_StoreStockQty,
        DC1_DOHStore,
        Store_DC1_cover_to_date,
        DC1_AvgSaleQty90D,
        DC1_AvgSaleQty30D,
        DC1_AvgSaleQty7D,
        [%Ratio_AvgSalesQty90D_DC1],
        DC1_Remain_StockQty,
        Current_DC1_DOH,
        Stock_DC1_cover_to_date,
        [Total-PO_qty_to_DC1],
        Min_delivery_date_to_DC1,
        [DC1_DOH(Stock+PO)],
        [Stock+PO_DC1_cover_to_date],
        DC2_ScmAssort,
        DC2_OOSAssort,
        DC2_CountOKROOS,
        DC2_PercOOS,
        DC2_StoreStockQty,
        DC2_DOHStore,
        Store_DC2_cover_to_date,
        DC2_AvgSaleQty90D,
        DC2_AvgSaleQty30D,
        DC2_AvgSaleQty7D,
        [%Ratio_AvgSalesQty90D_DC2],
        DC2_Remain_StockQty,
        Current_DC2_DOH,
        Stock_DC2_cover_to_date,
        [Total-PO_qty_to_DC2],
        Min_delivery_date_to_DC2,
        [DC2_DOH(Stock+PO)],
        [Stock+PO_DC2_cover_to_date],
        DC4_ScmAssort,
        DC4_OOSAssort,
        DC4_CountOKROOS,
        DC4_PercOOS,
        DC4_StoreStockQty,
        DC4_DOHStore,
        Store_DC4_cover_to_date,
        DC4_AvgSaleQty90D,
        DC4_AvgSaleQty30D,
        DC4_AvgSaleQty7D,
        [%Ratio_AvgSalesQty90D_DC4],
        DC4_Remain_StockQty,
        Current_DC4_DOH,
        Stock_DC4_cover_to_date,
        [Total-PO_qty_to_DC4],
        Min_delivery_date_to_DC4,
        [DC4_DOH(Stock+PO)],
        [Stock+PO_DC4_cover_to_date]
    FROM merged_df
    WHERE ([Group] != 'Discontinuous' or [Group] IS NULL)
    ORDER BY CJ_Item ASC
    """

    result_df = ps.sqldf(query, locals())
    

    # Rename All column for store to display a data this column is freezed
    columns_to_rename = {
        'Status': 'CJ_Status',
        'Group': 'SHM_Status',
        'Unit': 'Unit_of_Purchase',
        'LeadTime':'LeadTime(Days)',
        'Total_OOSAssort':'Total_ActiveAssort',
        'Total_CountOKROOS':'Total_StoreOOS',
        'Total_PercOOS':'Total_%StoreOOS'
        }
    
    # Rename the columns
    result_df.rename(columns=columns_to_rename, inplace=True)
    renamed_columns = [columns_to_rename[old_name] for old_name in columns_to_rename if old_name in result_df.columns]
    print(f"The following columns have been renamed: {columns_to_rename}")


    # Filter to show the data only all numeric column > 0
    numeric_cols = result_df.select_dtypes(include=[np.number]).columns
    result_df = result_df[(result_df[numeric_cols] != 0).any(axis=1)]

    # Format the date columns to display in Excel
    date_columns = [
        'First_SO_Date',
        'Min_delivery_date_to_DC',
        'Min_delivery_date_to_DC1',
        'Min_delivery_date_to_DC2',
        'Min_delivery_date_to_DC4',
        'Total_Store_cover_to_date',
        'Store_DC1_cover_to_date',
        'Store_DC2_cover_to_date',
        'Store_DC4_cover_to_date',
        'Stock_All_DC_Cover_to_date',
        'Stock_DC1_cover_to_date',
        'Stock_DC2_cover_to_date',
        'Stock_DC4_cover_to_date',
        'Stock+PO_All_DC_Cover_to_date',
        'Stock+PO_DC1_cover_to_date',
        'Stock+PO_DC2_cover_to_date',
        'Stock+PO_DC4_cover_to_date'
    ]

    for col in date_columns:
        if col in result_df.columns:
            result_df[col] = pd.to_datetime(result_df[col], errors='coerce')  # Ensure it's in datetime format
    

    # Drop SHM_Item = No shm item in access and Save deduplicated_df to excel
    #result_df.to_excel(r'D:\Data for Stock Report\Data_for_OOS_Log\11.Nov_24-11-2025.xlsx', index=False)

    # Handle duplicates 'CJ_Item' by using groupby
    def replace_cj_duplicate(group):
        special_products = ['20000408','20009203','20014191','20023778','20023779','20028264','20039186'] # deal with Sup = "ซัน ซัน"
        is_special_product = group['CJ_Item'].iloc[0] in special_products
        dc4_columns = [col for col in numeric_cols if 'DC4' in col] # get all column name contains 'DC4' and numeric type
        
        # if special product and only one row then return the group
        if is_special_product and len(group) == 1:
            return group
        
        if is_special_product: # if special product replace value with 0 for first rows contains 'DC4' in column name
            for col in numeric_cols:
                if col in dc4_columns or col == 'PC_Cartons':
                    continue
                if group[col].nunique() == 1:
                    group.iloc[1:, group.columns.get_loc(col)] = 0

            for col in dc4_columns:
                if group[col].nunique() == 1:
                    group.iloc[0, group.columns.get_loc(col)] = 0

        # Filter these 2 columns if both columns are 0 then drop the row, keep the first row
        #group = group[(group[['Remain_StockQty_AllDC', 'Total-PO_qty_to_DC']].ne(0).any(axis=1)) | (group.index == group.index.min())]

        return group
    
    # Apply this function to each group of 'CJ_Item'
    result_df = result_df.groupby('CJ_Item', group_keys=False).apply(replace_cj_duplicate).reset_index(drop=True)
    

    # Rename column for converting QTY to BOX QTY
    column_rename_mapping = {
        'Total_AvgSaleQty90D': 'Total_AvgSaleCTN_Last90D',
        'Total_AvgSaleQty30D': 'Total_AvgSaleCTN_Last30D',
        'Total_AvgSaleQty7D': 'Total_AvgSaleCTN_Last7D',
        'DC1_AvgSaleQty90D': 'DC1_AvgSaleCTN_Last90Days',
        'DC1_AvgSaleQty30D': 'DC1_AvgSaleCTN_Last30Days',
        'DC1_AvgSaleQty7D': 'DC1_AvgSaleCTN_Last7Days',
        'DC2_AvgSaleQty90D': 'DC2_AvgSaleCTN_Last90Days',
        'DC2_AvgSaleQty30D': 'DC2_AvgSaleCTN_Last30Days',
        'DC2_AvgSaleQty7D': 'DC2_AvgSaleCTN_Last7Days',
        'DC4_AvgSaleQty90D': 'DC4_AvgSaleCTN_Last90Days',
        'DC4_AvgSaleQty30D': 'DC4_AvgSaleCTN_Last30Days',
        'DC4_AvgSaleQty7D': 'DC4_AvgSaleCTN_Last7Days',
        'Total_StoreStockQty': 'Total_StoreStockCTN',
        'DC1_StoreStockQty': 'DC1_StoreStockCTN',
        'DC2_StoreStockQty': 'DC2_StoreStockCTN',
        'DC4_StoreStockQty': 'DC4_StoreStockCTN',
        'SO_Qty_last7D': 'SO_CTN_last7D',
        'Remain_StockQty_AllDC': 'Remain_CTN_AllDC',
        'DC1_Remain_StockQty': 'DC1_Remain_CTN',
        'DC2_Remain_StockQty': 'DC2_Remain_CTN',
        'DC4_Remain_StockQty': 'DC4_Remain_CTN',
        'Total-PO_qty_to_DC': 'Total-CTN_to_DC',
        'Total-PO_qty_to_DC1': 'Total-CTN_to_DC1',
        'Total-PO_qty_to_DC2': 'Total-CTN_to_DC2',
        'Total-PO_qty_to_DC4': 'Total-CTN_to_DC4'
    }

    # Process the rename
    modified_df = result_df.rename(columns=column_rename_mapping)

    # List of new columns that were just renamed
    renamed_columns = list(column_rename_mapping.values())
    
    # Loop through renamed column to perform division by PC_Cartons
    for col in renamed_columns:
        modified_df[col] = np.where(
            modified_df['PC_Cartons'] == 0,0,
            modified_df[col] / modified_df['PC_Cartons']
        )

    # Define current date and save path 
    current_date = datetime.now().strftime('%d-%m-%Y')
    save_directory = r'D:\Data for Stock Report\Completed Daily Stock Report'
    file_name = f"Sahamit Daily Stock Report_{current_date}.xlsx"
    save_path = os.path.join(save_directory, file_name)

    if not os.path.exists(save_directory):
        os.makedirs(save_directory)

    # Save the merged DataFrame
    try: 
        with pd.ExcelWriter(save_path, mode='w') as writer:
            result_df.to_excel(writer, sheet_name='Data by Qty', index=False)
            modified_df.to_excel(writer, sheet_name='Data by Cartons',index=False)

            # For checking Raw data
            #merged_df.to_excel(writer, sheet_name='Raw Data', index=False)

            # Load All sheet from Raw_PO_pending and save them together
            existing_file_path = r'D:\Data for Stock Report\Raw_PO_pending.xlsx'
            if os.path.exists(existing_file_path):
                workbook = load_workbook(existing_file_path)
                for sheet_name in workbook.sheetnames:
                    existing_df = pd.read_excel(existing_file_path,sheet_name=sheet_name)
                    existing_df.to_excel(writer,sheet_name=sheet_name,index=False)
            else:
                print(f"File not found: {existing_file_path}")

        print(f"Completed daily stock report has been saved to {save_path}")
    except Exception as e:
        print(f"An error occurred while saving the file: {e}")

if __name__ == "__main__":
    main()


C:\Users\Thanawit C\AppData\Local\Temp\ipykernel_15256\1494688372.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  owner_scm_df_selected.rename(columns={'Base Lead Time (Days)': 'LeadTime'}, inplace=True)
C:\Users\Thanawit C\AppData\Local\Temp\ipykernel_15256\1494688372.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  owner_scm_df_selected['CJ_Item'] = owner_scm_df_selected['CJ_Item'].astype(str)
C:\Users\Thanawit C\AppData\Local\Temp\ipykernel_15256\1494688372.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

Connection successful


C:\Users\Thanawit C\AppData\Local\Temp\ipykernel_15256\1494688372.py:101: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  access_df = pd.read_sql(query, conn)
C:\Users\Thanawit C\AppData\Local\Temp\ipykernel_15256\1494688372.py:102: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  query2_df = pd.read_sql(query2, conn)


Connection to Access database is closed.


C:\Users\Thanawit C\AppData\Local\Temp\ipykernel_15256\1494688372.py:247: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_df[f'Min_delivery_date_to_{dc}'] = merged_df[cols].min(axis=1)
C:\Users\Thanawit C\AppData\Local\Temp\ipykernel_15256\1494688372.py:247: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_df[f'Min_delivery_date_to_{dc}'] = merged_df[cols].min(axis=1)
C:\Users\Thanawit C\AppData\Local\Temp\ipykernel_15256\1494688372.py:247: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result 

The following columns have been renamed: {'Status': 'CJ_Status', 'Group': 'SHM_Status', 'Unit': 'Unit_of_Purchase', 'LeadTime': 'LeadTime(Days)', 'Total_OOSAssort': 'Total_ActiveAssort', 'Total_CountOKROOS': 'Total_StoreOOS', 'Total_PercOOS': 'Total_%StoreOOS'}


C:\Users\Thanawit C\AppData\Local\Temp\ipykernel_15256\1494688372.py:491: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_df = result_df.groupby('CJ_Item', group_keys=False).apply(replace_cj_duplicate).reset_index(drop=True)


Completed daily stock report has been saved to D:\Data for Stock Report\Completed Daily Stock Report\Sahamit Daily Stock Report_24-11-2025.xlsx
